## DATASET CLASS
_Represent a dataset of samples_

In [3]:
import os
import numpy as np

In [33]:
class Dataset(object):
    ''''''
    # Attributes
    
    # Initializer
    def __init__(self,dataset_location,  site):
        self.__dataset_location = dataset_location
        self.__site = site
    
    # 
    def get_site(self):
        return self.__site
    
    def get_dataset_location(self):
        return self.__dataset_location
    
    # 
    def get_samples_name(self):
        '''Get all samples datafiles
        params:
            awd_event
            site
        returns:
            datafiles
        '''
        data_location = os.path.join(self.__dataset_location, self.__site, self.__site+'_data')
        print(data_location)
        files = None
        if os.path.exists(data_location):
            files = [ file for file in os.listdir(data_location) if file.endswith('.vr2')] # only select .vr2 file
        return files

    def get_random_sample(self):
        '''Select a random datafile
        params:
            awd_event
            site
        returns:
            file name
        '''
        files = self.get_samples_name()
        return files[np.random.randint(len(files))]

In [36]:
loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data', 'awdEvents1')
my_dataset = Dataset(loc,'marion')
print(my_dataset.get_random_sample())

/home/othniel/wdml/data/awdEvents1/marion/marion_data
2013-07-11UT05:10:01.14531938.marion.vr2
